In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import re
import rasterio
import spenv
import sys
sys.path.insert(0, '../src/')
import preproc

In [2]:
directory = 'raw'
files = [directory + "/" + f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.startswith('raw')]

def extract_number(s):
    match = re.search(r'\d+', s)
    return int(match.group()) if match else float('inf')

sorted_files = sorted(files, key=extract_number)
print(str(len(sorted_files)) + ' files found.')

81 files found.


In [3]:
rasters = []
for i in sorted_files:
    temp = rasterio.open(i)
    rasters.append(temp.read())
    temp.close()

In [4]:
cube = np.concatenate(rasters)

In [5]:
stdev = np.std(cube)
avg = np.mean(cube)
scaled = (cube - avg) / stdev

In [6]:
for i in range(scaled.shape[0]):
    file_name = 'scaled/scaled_' + str(i) + '.tif'
    preproc.save_with_rio(file_name, scaled[i], temp)

In [9]:
optimal = np.ndarray(scaled.shape)
size = optimal.shape[1] * optimal.shape[2]
counter = 0

start = time.time()
for i in range(0, optimal.shape[1]):
    for j in range(0, optimal.shape[2]):
        counter += 1
        aux = scaled[:, i, j].reshape((-1, 1))
        optimal[:, i, j] = spenv.spec_opt(aux, lambda t: np.abs(t)**5)
        print(str(counter) + "/" + str(size), end="\r")

end = time.time()
print("Time: " + str(end-start))

Time: 918.86110663414


In [13]:
for i in range(optimal.shape[0]):
    file_name = 'optimized/optimized_' + str(i) + '.tif'
    preproc.save_with_rio(file_name, optimal[i], temp)